In [2]:
from catboost import CatBoostClassifier

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/1c/e1/78e635a1e5f0066bd02a1ecfd658ad09fe30d275c65c2d0dd76fe253e648/catboost-1.2.7-cp311-cp311-win_amd64.whl.metadata
  Using cached catboost-1.2.7-cp311-cp311-win_amd64.whl.metadata (1.2 kB)
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/02/65/ad2bc85f7377f5cfba5d4466d5474423a3fb7f6a97fd807c06f92dd3e721/plotly-6.0.1-py3-none-any.whl.metadata
  Using cached plotly-6.0.1-py3-none-any.whl.metadata (6.7 kB)
  Obtaining dependency information for narwhals>=1.15.1 from https://files.pythonhosted.org/packages/f9/c0/fb39bd876ea2fd9509343d643690cd2f9715e6a77271e7c7b26f1eea70c1/narwhals-1.31.0-py3-n


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
model = CatBoostClassifier(iterations=100) # number of trees
model.fit(X, y, cat_features=cat_features, verbose=10) # categorical features

In [ ]:
from catboost import Pool 
pool=Pool(data=X,label=y,cat_features=cat_features)

In [ ]:
from sklearn.model_selection import train_test_split

data = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_validation, y_train, y_validation = data

train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_features
)

validation_pool = Pool(
    data=X_validation,
    label=y_validation,
    cat_features=cat_features
)


In [ ]:
# CrossEntropy for probabilities in target.

model = CatBoostClassifier(
    iterations=5,
    learning_rate=0.1,
    loss_function='CrossEntropy'
)

model.fit(train_pool, eval_set=validation_pool, verbose=False)

print("Model is fitted: {}".format(model.is_fitted()))
print("Model params: {}".format(model.get_params()))


In [ ]:
model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.5,
    custom_loss=['AUC', 'Accuracy'], 
    eval_metric="AUC",
    early_stopping=20
)

model.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=False,
    plot=True
)
#select treshold

In [ ]:
from catboost import cv

params = {
    'loss_function': 'Logloss',
    'iterations': 100,
    'custom_loss': 'AUC',
    'learning_rate': 0.2
}
# by default it stratifies
cv_data = cv(
    params=params,
    pool=train_pool,
    fold_count=3,
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    stratified=True,
    verbose=False
)

cv_data.head(10)

best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print(
    "Best validation Logloss score, not stratified: {:.4f}±{:.4f} on step {}".format(
        best_value, 
        cv_data['test-Logloss-std'][best_iter], 
        best_iter
    )
)


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.001, 0.01, 0.5]
}

clf = CatBoostClassifier(
    iterations=50,
    cat_features=cat_features,
    verbose=20
)

grid_search = GridSearchCV(clf, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_.get_params()

In [ ]:
tunned_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3,
    random_strength=1,
    bagging_temperature=1
)

tunned_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    verbose=False,
    eval_set=(X_validation, y_validation),
    plot=True
)